In [154]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.16.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 222.00 kB/s
geopy-1.16.0-p 100% |################################| Time: 0:00:00 341.98 kB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.1.0-py_0 conda-forge
    branca:  0.3.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

branca-0.3.0-p 100% |################################| Time: 0:00:00 306.36 kB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00 380.40 kB/s
altair-2.1.0-p 100% |###########################

In [2]:
wikipediaPage = requests.get("https://en.wikipedia.org/wiki/Subdivisions_of_Z%C3%BCrich")

In [31]:
# import it in a soup object
soup = BeautifulSoup(wikipediaPage.content, 'html5lib')

# get the table with the needed info
table = soup.find_all('table')[1]

# extract all <tr> from the table
trs = table.find_all('tr')

In [144]:
# function to parse the <tr> and return an entry for the dataframe : return Borough, Neighborhood
def parseTableRow(tr, index):
    borough = 'District ' + str(index)
    
    neighborhood = ''

    td = tr.find_next('td')
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    td = td.next_sibling
    
    for string in td.stripped_strings:
        if (neighborhood != ''):
            neighborhood = neighborhood + ', '
        neighborhood = neighborhood + string
    
    return borough, neighborhood

In [150]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood


In [151]:
# fill the dataframe
index = 1
for tr in trs:
    if 'District ' in str(tr):
        data = parseTableRow(tr, index)
        neighborhoods = neighborhoods.append({'Borough': data[0], 'Neighborhood': data[1]}, ignore_index=True)
        index = index + 1

In [152]:
neighborhoods

,Borough,Neighborhood
0,District 1,"Rathaus, Hochschulen, Lindenhof, City"
1,District 2,"Wollishofen, Leimbach, Enge"
2,District 3,"Alt-Wiedikon, Friesenberg, Sihlfeld"
3,District 4,"Werd, Langstrasse, Hard"
4,District 5,"Gewerbeschule, Escher Wyss"
5,District 6,"Unterstrass, Oberstrass"
6,District 7,"Fluntern, Hottingen, Hirslanden, Witikon"
7,District 8,"Seefeld, Mühlebach, Weinegg"
8,District 9,"Albisrieden, Altstetten"
9,District 10,"Höngg, Wipkingen"


In [168]:
# The code was removed by Watson Studio for sharing.

In [173]:
baseUrl = 'https://maps.googleapis.com/maps/api/geocode/json?'

In [180]:
def getLocation(district):
    """ 
    Search for information for place Zürich<district>
    return the found place latitude and longitude (return latitude, longitude)
    """
    url = (baseUrl+'&address={}').format('Zürich'+district)
    response = requests.get(url).json() # get response
    geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
    latitude = geographical_data['lat']
    longitude = geographical_data['lng']
    return latitude, longitude

In [190]:
nh = neighborhoods.copy()

latitudeCln = []
longitudeCln = []
for index, row in nh.iterrows():
    lat, long = getLocation(row[0])
    latitudeCln.append(lat)
    longitudeCln.append(long)

nh['Latitude'] = latitudeCln
nh['Longitude'] = longitudeCln

nh.shape

(12, 4)

In [191]:
nh

,Borough,Neighborhood,Latitude,Longitude
0,District 1,"Rathaus, Hochschulen, Lindenhof, City",47.374156,8.539632
1,District 2,"Wollishofen, Leimbach, Enge",47.343046,8.527225
2,District 3,"Alt-Wiedikon, Friesenberg, Sihlfeld",47.364101,8.509246
3,District 4,"Werd, Langstrasse, Hard",47.380756,8.517289
4,District 5,"Gewerbeschule, Escher Wyss",47.387768,8.522438
5,District 6,"Unterstrass, Oberstrass",47.393446,8.545447
6,District 7,"Fluntern, Hottingen, Hirslanden, Witikon",47.368398,8.583532
7,District 8,"Seefeld, Mühlebach, Weinegg",47.387768,8.522438
8,District 9,"Albisrieden, Altstetten",47.386046,8.479356
9,District 10,"Höngg, Wipkingen",47.404459,8.497033


In [155]:
address = 'Zürich, Switzerland'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zürich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zürich are 47.3723957, 8.5423216.


In [192]:
# make a copy of the dataframe to get it simply back if needed
nhz = nh.copy()

In [193]:
# create map of Zürich using latitude and longitude values
map_zh = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(nhz['Latitude'], nhz['Longitude'], nhz['Borough'], nhz['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh)  
    
map_zh

In [194]:
# The code was removed by Watson Studio for sharing.

In [195]:
# function to repeat the exploring process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [196]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [197]:
zh_venues = getNearbyVenues(names=nhz['Neighborhood'], latitudes=nhz['Latitude'], longitudes=nhz['Longitude'])

Rathaus, Hochschulen, Lindenhof, City
Wollishofen, Leimbach, Enge
Alt-Wiedikon, Friesenberg, Sihlfeld
Werd, Langstrasse, Hard
Gewerbeschule, Escher Wyss
Unterstrass, Oberstrass
Fluntern, Hottingen, Hirslanden, Witikon
Seefeld, Mühlebach, Weinegg
Albisrieden, Altstetten
Höngg, Wipkingen
Affoltern, Oerlikon, Seebach
Saatlen, Schwamendingen Mitte, Hirzenbach


In [198]:
print(zh_venues.shape)
zh_venues.head()

(378, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rathaus, Hochschulen, Lindenhof, City",47.374156,8.539632,Lindenhof,47.373005,8.540883,Scenic Lookout
1,"Rathaus, Hochschulen, Lindenhof, City",47.374156,8.539632,Widder Bar,47.372415,8.539863,Hotel Bar
2,"Rathaus, Hochschulen, Lindenhof, City",47.374156,8.539632,Dachterrasse Hiltl,47.375686,8.539650,Vegetarian / Vegan Restaurant
3,"Rathaus, Hochschulen, Lindenhof, City",47.374156,8.539632,Widder Hotel,47.372449,8.539973,Hotel
4,"Rathaus, Hochschulen, Lindenhof, City",47.374156,8.539632,Apple Bahnhofstrasse,47.375018,8.538677,Electronics Store


In [199]:
zh_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Affoltern, Oerlikon, Seebach",4,4,4,4,4,4
"Albisrieden, Altstetten",6,6,6,6,6,6
"Alt-Wiedikon, Friesenberg, Sihlfeld",9,9,9,9,9,9
"Fluntern, Hottingen, Hirslanden, Witikon",3,3,3,3,3,3
"Gewerbeschule, Escher Wyss",99,99,99,99,99,99
"Höngg, Wipkingen",11,11,11,11,11,11
"Rathaus, Hochschulen, Lindenhof, City",100,100,100,100,100,100
"Saatlen, Schwamendingen Mitte, Hirzenbach",16,16,16,16,16,16
"Seefeld, Mühlebach, Weinegg",99,99,99,99,99,99


In [200]:
print('There are {} uniques categories.'.format(len(zh_venues['Venue Category'].unique())))

There are 107 uniques categories.


In [201]:
# one hot encoding
zh_onehot = pd.get_dummies(zh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zh_onehot['Neighborhood'] = zh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [zh_onehot.columns[-1]] + list(zh_onehot.columns[:-1])
zh_onehot = zh_onehot[fixed_columns]

zh_onehot.head()

,Neighborhood,Accessories Store,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bakery,Bar,Beer Garden,Bistro,Bookstore,Boutique,Burger Joint,Burrito Place,Bus Station,Café,Cambodian Restaurant,Candy Store,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Furniture / Home Store,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Kids Store,Light Rail Station,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Multiplex,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Pizza Place,Planetarium,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Sandwich Place,Scenic Lookout,Shoe Store,Shopping Mall,Snack Place,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tech Startup,Thai Restaurant,Theater,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Rathaus, Hochschulen, Lindenhof, City",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rathaus, Hochschulen, Lindenhof, City",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rathaus, Hochschulen, Lindenhof, City",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,"Rathaus, Hochschulen, Lindenhof, City",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rathaus, Hochschulen, Lindenhof, City",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [205]:
zh_grouped = zh_onehot.groupby('Neighborhood').sum().reset_index()
zh_grouped

,Neighborhood,Accessories Store,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Automotive Shop,Bakery,Bar,Beer Garden,Bistro,Bookstore,Boutique,Burger Joint,Burrito Place,Bus Station,Café,Cambodian Restaurant,Candy Store,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Furniture / Home Store,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Kids Store,Light Rail Station,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Multiplex,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paella Restaurant,Park,Pedestrian Plaza,Pizza Place,Planetarium,Plaza,Pool,Pool Hall,Pub,Restaurant,River,Rock Club,Sandwich Place,Scenic Lookout,Shoe Store,Shopping Mall,Snack Place,Southern / Soul Food Restaurant,Spanish Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tech Startup,Thai Restaurant,Theater,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Affoltern, Oerlikon, Seebach",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,"Albisrieden, Altstetten",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0
2,"Alt-Wiedikon, Friesenberg, Sihlfeld",0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Fluntern, Hottingen, Hirslanden, Witikon",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,"Gewerbeschule, Escher Wyss",1,0,2,0,3,0,3,5,1,2,0,0,1,0,0,4,1,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,2,0,1,0,0,1,0,1,2,0,1,0,0,1,2,2,3,0,1,1,1,3,1,1,1,0,0,1,1,1,0,1,1,6,1,0,0,0,0,0,1,0,3,0,3,0,1,2,3,1,1,1,0,0,2,1,1,1,0,1,0,2,1,1,2,1,1,0,0,0,0,0,1
5,"Höngg, Wipkingen",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,2,0,0,0,0,0
6,"Rathaus, Hochschulen, Lindenhof, City",0,1,0,2,2,1,2,8,0,0,1,1,0,1,0,4,0,1,0,1,1,3,1,0,0,0,1,5,2,2,0,0,0,1,0,0,0,0,1,0,0,2,0,1,0,1,2,1,0,1,5,1,0,0,0,4,0,0,1,0,0,4,0,0,0,0,0,0,0,1,1,0,0,0,1,2,0,1,2,0,0,0,4,0,0,0,1,1,1,0,0,0,1,1,1,7,0,0,1,0,1,0,4,2,1,1,0
7,"Saatlen, Schwamendingen Mitte, Hirzenbach",0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,1,0,0,1,0,0,3,0,0,0,0,0
8,"Seefeld, Mühlebach, Weinegg",1,0,2,0,3,0,3,5,1,2,0,0,1,0,0,4,1,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,1,1,2,0,1,0,0,1,0,1,2,0,1,0,0,1,2,2,3,0,1,1,1,3,1,1,1,0,0,1,1,1,0,1,1,6,1,0,0,0,0,0,1,0,3,0,3,0,1,2,3,1,1,1,0,0,2,1,1,1,0,1,0,2,1,1,2,1,1,0,0,0,0,0,1
9,"Unterstrass, Oberstrass",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0


In [206]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [207]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = zh_grouped['Neighborhood']

for ind in np.arange(zh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(zh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Affoltern, Oerlikon, Seebach",Farm,Swiss Restaurant,Forest,Plaza,Yoga Studio,Food & Drink Shop,Department Store,Dessert Shop,Diner,Discount Store
1,"Albisrieden, Altstetten",Pool,Hotel,Tram Station,Swiss Restaurant,Supermarket,Bus Station,Fast Food Restaurant,Department Store,Dessert Shop,Diner
2,"Alt-Wiedikon, Friesenberg, Sihlfeld",Bus Station,Discount Store,Asian Restaurant,Supermarket,Restaurant,Lounge,Beer Garden,Farm,Food & Drink Shop,Fast Food Restaurant
3,"Fluntern, Hottingen, Hirslanden, Witikon",Cosmetics Shop,Other Great Outdoors,Swiss Restaurant,Food Court,Dessert Shop,Diner,Discount Store,Doner Restaurant,Eastern European Restaurant,Electronics Store
4,"Gewerbeschule, Escher Wyss",Nightclub,Bar,Café,Italian Restaurant,Asian Restaurant,Bakery,Restaurant,Pizza Place,Hotel,Plaza
5,"Höngg, Wipkingen",Grocery Store,Tram Station,Mexican Restaurant,Fast Food Restaurant,Indian Restaurant,Steakhouse,Bakery,Food & Drink Shop,Supermarket,Falafel Restaurant
6,"Rathaus, Hochschulen, Lindenhof, City",Bar,Swiss Restaurant,Hotel,Department Store,Lounge,Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Café,Cocktail Bar
7,"Saatlen, Schwamendingen Mitte, Hirzenbach",Tram Station,Plaza,Bus Station,Supermarket,Italian Restaurant,Fast Food Restaurant,Shopping Mall,Swiss Restaurant,Light Rail Station,Thai Restaurant
8,"Seefeld, Mühlebach, Weinegg",Nightclub,Bar,Café,Italian Restaurant,Asian Restaurant,Bakery,Restaurant,Pizza Place,Hotel,Plaza
9,"Unterstrass, Oberstrass",Doner Restaurant,Café,Falafel Restaurant,Middle Eastern Restaurant,Snack Place,Paella Restaurant,Kids Store,Supermarket,Swiss Restaurant,Scenic Lookout
